In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]='3'
os.environ["CUDA_LAUNCH_BLOCKING"]='1'

import glob
import pandas as pd
import numpy as np
import cudf
import cupy
import gc
from datetime import datetime

from util import compute_rce_fast

DP = len(os.environ["CUDA_VISIBLE_DEVICES"].split(','))>1
DP

False

In [2]:
import cupy as cp
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import nvtabular as nvt
from nvtabular.loader.torch import TorchAsyncItr, DLDataLoader
from nvtabular.framework_utils.torch.models import Model
from nvtabular.framework_utils.torch.utils import process_epoch

import torch
from torch import nn
torch.__version__

'1.7.1+cu101'

## model

In [3]:
class ConcatenatedEmbeddings(torch.nn.Module):
    """Map multiple categorical variables to concatenated embeddings.
    Args:
        embedding_table_shapes: A dictionary mapping column names to
            (cardinality, embedding_size) tuples.
        dropout: A float.
    Inputs:
        x: An int64 Tensor with shape [batch_size, num_variables].
    Outputs:
        A Float Tensor with shape [batch_size, embedding_size_after_concat].
    """

    def __init__(self, embedding_table_shapes, dropout=0.0):
        super().__init__()
        self.embedding_layers = torch.nn.ModuleList(
            [
                torch.nn.Embedding(cat_size, emb_size) #, sparse=(cat_size > 1e5))
                for cat_size, emb_size in embedding_table_shapes.values()
            ]
        )
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x):
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
            
        # first two cat columns (a_user and b_user) share same emb table            
        x = [self.embedding_layers[0](x[:,0])] + [layer(x[:, i+1]) for i, layer in enumerate(self.embedding_layers)] 
        x = torch.cat(x, dim=1)
        x = self.dropout(x)
        return x

In [4]:
import torch.nn as nn

sigmoid = nn.Sigmoid()

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Swish_Module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)


from transformers import AutoTokenizer, AutoModel

bert_type = 'distilbert-base-multilingual-cased'

tokenizer = AutoTokenizer.from_pretrained(bert_type)

class Net(nn.Module):
    def __init__(self, num_features, layers, embedding_table_shapes, dropout=0.2, bert_type=None, gru_dim=128, emb_dim=768):
        super(Net, self).__init__()
        self.dropout = dropout
        self.initial_cat_layer = ConcatenatedEmbeddings(embedding_table_shapes, dropout=dropout)
        embedding_size = sum(emb_size for _, emb_size in embedding_table_shapes.values())
        layers = [layers] if type(layers) is int else layers
        layers = [num_features + gru_dim + embedding_size + 128 + 128] + layers
        self.use_bert = True
        self.embed = AutoModel.from_pretrained(bert_type).embeddings.word_embeddings  
        assert emb_dim == self.embed.embedding_dim
#             self.reduce_dim = nn.Linear(self.embed.embedding_dim, 256)
#             self.embed = nn.Embedding(119547, emb_dim)
#         layers[0] += gru_dim
        self.lstm = nn.GRU(emb_dim, gru_dim, batch_first=True, bidirectional=False)    
#             self.lstm = nn.Linear(self.embed.embedding_dim, gru_dim)

        self.fn_layers = nn.ModuleList(
                            nn.Sequential(
                                nn.Dropout(p=dropout),
                                nn.Linear(layers[i], layers[i+1]),
                                nn.BatchNorm1d(layers[i+1]),
                                Swish_Module(),
                            )  for i in range(len(layers) -1)
                         )        
        self.fn_last = nn.Linear(layers[-1],4)
        
    def forward(self, x_cat, x_cont, bert_tok):
        a_emb = self.initial_cat_layer.embedding_layers[0](x_cat[:,0])
        b_emb = self.initial_cat_layer.embedding_layers[0](x_cat[:,1])
        mf = a_emb * b_emb        
        
        x_cat = self.initial_cat_layer(x_cat)
        bert_tok = self.embed(bert_tok)#.mean(dim=1)
#             bert_tok = self.reduce_dim(bert_tok)
        lstm_out = self.lstm(bert_tok)[0][:,-1]
        output = torch.cat([x_cont, lstm_out, x_cat, mf],dim=1)
        for layer in self.fn_layers:
            output = layer(output)
        logit = self.fn_last(output)
        return logit

## scheduler

In [5]:
from warmup_scheduler import GradualWarmupScheduler
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import warnings; warnings.simplefilter('ignore')

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs] 

## train loop

In [6]:
criterion = nn.BCEWithLogitsLoss()

def train_epoch(model, loader, optimizer, scaler):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for batch in bar:
        x_cat, x_cont, text_tok, targets = batch
        
        x_cat = x_cat.cuda()
        x_cont = x_cont.cuda()
        text_tok = text_tok.cuda()
        targets = targets.cuda()

        optimizer.zero_grad()
#         optimizer2.zero_grad()

        if use_torch_amp:
            with amp.autocast():
                logits = model(x_cat, x_cont, text_tok)
#                 logits = model(data)
            loss = criterion(logits, targets)       
            
            scaler.scale(loss).backward()

            # You can choose which optimizers receive explicit unscaling, if you
            # want to inspect or modify the gradients of the params they own.
            scaler.unscale_(optimizer)
#             scaler.unscale_(optimizer2)

            scaler.step(optimizer)
#             scaler.step(optimizer2)

            scaler.update()            
            
        elif use_amp:
            logits = model(x_cat, x_cont, text_tok)
#             logits = model(data)
            loss = criterion(logits, targets)
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            optimizer.step()
        else:
            logits = model(x_cat, x_cont, text_tok)
#             logits = model(data)
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

        loss_np = loss.item()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-50:]) / min(len(train_loss), 50)
        bar.set_description('loss: %.4f, smth: %.4f' % (loss_np, smooth_loss))

    return np.mean(train_loss)

def valid_epoch(model, loader):

    model.eval()
    val_loss = []
    LOGITS = []
    TARGETS = []
    with torch.no_grad():
        for batch in tqdm(loader):
            x_cat, x_cont, text_tok, targets = batch

            x_cat = x_cat.cuda()
            x_cont = x_cont.cuda()
            text_tok = text_tok.cuda()
            targets = targets.cuda()
        
            logits = model(x_cat, x_cont, text_tok)
#             logits = model(data)
            loss = criterion(logits, targets)
            val_loss.append(loss.item())
            LOGITS.append(logits.cpu())
            TARGETS.append(targets.cpu())
            
    LOGITS = torch.cat(LOGITS)
    TARGETS = torch.cat(TARGETS)
    rce = {}
    for i in range(4):
        rce[label_names[i]] = compute_rce_fast(cp.asarray(LOGITS[:,i].sigmoid()),cp.asarray(TARGETS[:,i])).get()            
    mean_rce = np.mean([v for k,v in rce.items()])
            
    val_loss = np.mean(val_loss)

    return val_loss, rce, mean_rce

# NVT loader

In [7]:
label_names = sorted(['reply', 'retweet', 'retweet_comment', 'like'])
CAT_COLUMNS = ['a_user_id','b_user_id','language','media','tweet_type']
NUMERIC_COLUMNS = ['a_follower_count',
                     'a_following_count',
                     'a_is_verified',
                     'b_follower_count',
                     'b_following_count',
                     'b_is_verified',
                     'b_follows_a',
                     'tw_len_media',
                     'tw_len_photo',
                     'tw_len_video',
                     'tw_len_gif',
                     'tw_len_quest',
                     'tw_len_token',
                     'tw_count_capital_words',
                     'tw_count_excl_quest_marks',
                     'tw_count_special1',
                     'tw_count_hash',
                     'tw_last_quest',
                     'tw_len_retweet',
                     'tw_len_rt',
                     'tw_count_at',
                     'tw_count_words',
                     'tw_count_char',
                     'tw_rt_count_words',
                     'tw_rt_count_char',
                     'len_hashtags',
                     'len_links',
                     'len_domains',
                     'a_ff_rate',
                     'b_ff_rate',
                     'ab_fing_rate',
                     'ab_fer_rate',
                     'a_age',
                     'b_age',
                     'ab_age_dff',
                     'ab_age_rate']
len(NUMERIC_COLUMNS)

36

In [8]:
def read_norm_merge(path, split='train'):
    ddf = pd.read_parquet(path)

    ddf['quantile'] = 0
    quantiles = [92, 216, 442, 1064]
    for i, quant in enumerate(quantiles):
        ddf['quantile'] = (ddf['quantile']+(ddf['a_follower_count']>quant).astype('int8')).astype('int8')

    ddf['date'] = pd.to_datetime(ddf['timestamp'], unit='s')
    
    VALID_DOW = '2021-02-18'
    if split=='train':
        ddf = ddf[ddf['date']<pd.to_datetime(VALID_DOW)].reset_index(drop=True)
    elif split=='valid':
        ddf = ddf[ddf['date']>=pd.to_datetime(VALID_DOW)].reset_index(drop=True)    
    else:
        pass
    
    ddf['a_ff_rate'] = (ddf['a_following_count'] / ddf['a_follower_count']).astype('float32')
    ddf['b_ff_rate'] = (ddf['b_follower_count']  / ddf['b_following_count']).astype('float32')
    ddf['ab_fing_rate'] = (ddf['a_following_count'] / ddf['b_following_count']).astype('float32')
    ddf['ab_fer_rate'] = (ddf['a_follower_count'] / (1+ddf['b_follower_count'])).astype('float32')
    ddf['a_age'] = ddf['a_account_creation'].astype('int16') + 128
    ddf['b_age'] = ddf['b_account_creation'].astype('int16') + 128
    ddf['ab_age_dff'] = ddf['b_age'] - ddf['a_age']
    ddf['ab_age_rate'] = ddf['a_age']/(1+ddf['b_age'])

    ## Normalize
    for col in NUMERIC_COLUMNS:
        if col == 'tw_len_quest':
            ddf[col] = np.clip(ddf[col].values,0,None)
        if ddf[col].dtype == 'uint16':
            ddf[col].astype('int32')

        if col == 'ab_age_dff':
            ddf[col] = ddf[col] / 256.            
        elif 'int' in str(ddf[col].dtype) or 'float' in str(ddf[col].dtype):    
            ddf[col] = np.log1p(ddf[col])

        if ddf[col].dtype == 'float64':
            ddf[col] = ddf[col].astype('float32') 

    ## get categorical embedding id        
    for col in CAT_COLUMNS:
        ddf[col] = ddf[col].astype('float')
        if col in ['a_user_id','b_user_id']:
            mapping_col = 'a_user_id_b_user_id'
        else:
            mapping_col = col
        mapping = pd.read_parquet(f'/raid/recsys_pre_TE_w_tok/workflow_232parts_joint_thr25/categories/unique.{mapping_col}.parquet').reset_index()
        mapping.columns = ['index',col]
        ddf = ddf.merge(mapping, how='left', on=col).drop(columns=[col]).rename(columns={'index':col})
        ddf[col] = ddf[col].fillna(0).astype('int')        

    label_names = ['reply', 'retweet', 'retweet_comment', 'like']
    DONT_USE = ['timestamp','a_account_creation','b_account_creation','engage_time',
                'fold', 'dt_dow', 'a_account_creation', 
                'b_account_creation', 'elapsed_time', 'links','domains','hashtags','id', 'date', 'is_train', 
                'tw_hash0', 'tw_hash1', 'tw_hash2', 'tw_http0', 'tw_uhash', 'tw_hash', 'tw_word0', 
                'tw_word1', 'tw_word2', 'tw_word3', 'tw_word4', 'dt_minute', 'dt_second',
               'dt_day', 'group', 'text', 'tweet_id', 'tw_original_user0', 'tw_original_user1', 'tw_original_user2',
                'tw_rt_user0', 'tw_original_http0', 'tw_tweet',]
    DONT_USE = [c for c in ddf.columns if c in DONT_USE]
    gc.collect(); gc.collect()
    
    return ddf.drop(columns=DONT_USE)

In [16]:
PATHS = sorted(glob.glob('/raid/recsys/train_proc3/*.parquet'))
len(PATHS)

232

In [10]:
# for col in NUMERIC_COLUMNS:
#     print(col)
#     plt.hist(train[col].values, bins=50)
#     plt.title(col)
# #     print(ddf[col].describe())
#     plt.show()

In [8]:
import torch
from torch.utils.data import Dataset,DataLoader

class AllDataset(Dataset):
    def __init__(self, df, max_len_txt, NUMERIC_COLUMNS, CAT_COLUMNS):
        self.X = df[NUMERIC_COLUMNS].values
        self.X_cat = df[CAT_COLUMNS].values
        self.labels = df[label_names].values
        self.text_tokens = df.text_tokens.values
        self.max_len_txt = max_len_txt
    def __len__(self):
        return self.labels.shape[0]
    def __getitem__(self, index):        
#         text = tokenizer.decode([int(token_id) for token_id in self.text_tokens[index][4:-4].split('\t')]) # [4:-4] is to remove [CLS] and [SEP]
#         inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_len_txt, return_tensors='pt')['input_ids'].squeeze()
        inputs = [int(token_id) for token_id in self.text_tokens[index].split('\t')][:self.max_len_txt]
        if len(inputs) < self.max_len_txt:
            inputs += [0]*(self.max_len_txt-len(inputs))
        return self.X_cat[index], self.X[index].astype(np.float32), torch.tensor(inputs), self.labels[index].astype(np.float32)

In [9]:
gru_dim=128
max_len_txt=48
emb_dim=768
lr = 1e-4 
ep = 46   
BATCH_SIZE = 1024
num_workers = 16
use_torch_amp = True
import torch.cuda.amp as amp
use_amp = False

model_name = 'MF_len48_joint_thr25_3weeks'

In [10]:
len(NUMERIC_COLUMNS)

36

In [14]:
%%time
train_lst = []
for path in PATHS[:10]:
    train_lst.append(read_norm_merge(path, 'valid'))
valid = pd.concat(train_lst)
gc.collect()

valid_dataset = AllDataset(valid, max_len_txt, NUMERIC_COLUMNS, CAT_COLUMNS)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers) 
valid.shape, len(valid_loader)

CPU times: user 4min 58s, sys: 49.7 s, total: 5min 47s
Wall time: 5min 35s


((10324907, 47), 10083)

In [15]:
# train_parts_order = np.concatenate([np.random.permutation(232)])
train_parts_order = np.array([ 46, 111, 208, 230,   3,  22, 227, 153,  78,  52,  20, 185,   6,
        130, 177,  83,  97, 194,  24, 187,  93,  59, 217, 180, 129,  62,
          1,  43, 229, 102, 196,  50,   4,  12, 114,  70,  18,  91,  71,
        190, 174,  23,  63,  89, 188,  16, 104,  67,  39, 225, 176,  28,
        198,   2,  76, 166, 216, 116, 199, 113, 107, 201,  64, 115,   8,
        171,  44, 218, 158, 181,  79,  47, 155, 159, 164, 109,  56, 106,
        122, 203, 144,  14, 163, 124, 110, 126,  80,  77,  94, 135,  33,
        134, 224, 145, 172, 191,  60, 148, 215, 212, 219,  35, 167,  37,
        132, 182, 228,  75,  87, 156, 137,  74,  29,  95, 118,  90, 222,
         19,  57, 162, 105, 223, 210, 140,  10,  72, 152, 183, 170,  51,
         82, 117,  13, 211, 120,  81, 160,  27, 200, 128, 169, 213, 179,
         42,  11, 143,  15, 209, 151,  48, 207, 112, 119, 231, 175,   0,
        146, 154,  68, 197,  21, 206, 125, 192,  31,  86, 138,  36, 108,
        103,  58, 142,  54,  98,  99, 127, 214,   7,  92, 121, 202, 141,
        150,  88,  53,  38, 139, 147, 131,  66,  40,  26, 123,  73, 100,
        165, 186, 149, 205,   5, 189,  25,  32, 133, 101, 204, 178, 193,
        136,  84, 161,  30, 221,  65,  85,  41,  17,  61,  45, 173, 195,
          9, 184,  55,  49, 168,  69,  34,  96, 157, 226, 220])
train_parts_order, train_parts_order.shape

(array([ 46, 111, 208, 230,   3,  22, 227, 153,  78,  52,  20, 185,   6,
        130, 177,  83,  97, 194,  24, 187,  93,  59, 217, 180, 129,  62,
          1,  43, 229, 102, 196,  50,   4,  12, 114,  70,  18,  91,  71,
        190, 174,  23,  63,  89, 188,  16, 104,  67,  39, 225, 176,  28,
        198,   2,  76, 166, 216, 116, 199, 113, 107, 201,  64, 115,   8,
        171,  44, 218, 158, 181,  79,  47, 155, 159, 164, 109,  56, 106,
        122, 203, 144,  14, 163, 124, 110, 126,  80,  77,  94, 135,  33,
        134, 224, 145, 172, 191,  60, 148, 215, 212, 219,  35, 167,  37,
        132, 182, 228,  75,  87, 156, 137,  74,  29,  95, 118,  90, 222,
         19,  57, 162, 105, 223, 210, 140,  10,  72, 152, 183, 170,  51,
         82, 117,  13, 211, 120,  81, 160,  27, 200, 128, 169, 213, 179,
         42,  11, 143,  15, 209, 151,  48, 207, 112, 119, 231, 175,   0,
        146, 154,  68, 197,  21, 206, 125, 192,  31,  86, 138,  36, 108,
        103,  58, 142,  54,  98,  99, 127, 214,   7

In [11]:
model = Net(len(NUMERIC_COLUMNS), layers=[1024,256,64], 
            embedding_table_shapes={'a_user_id_b_user_id': (8244536, 128), 'language': (67, 16), 'media': (15, 16), 'tweet_type': (4, 16)},
            bert_type=bert_type).cuda()

for param in model.embed.parameters():
    param.requires_grad = False

model    

Net(
  (initial_cat_layer): ConcatenatedEmbeddings(
    (embedding_layers): ModuleList(
      (0): Embedding(8244536, 128)
      (1): Embedding(67, 16)
      (2): Embedding(15, 16)
      (3): Embedding(4, 16)
    )
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (embed): Embedding(119547, 768, padding_idx=0)
  (lstm): GRU(768, 128, batch_first=True)
  (fn_layers): ModuleList(
    (0): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=596, out_features=1024, bias=True)
      (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Swish_Module()
    )
    (1): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=1024, out_features=256, bias=True)
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Swish_Module()
    )
    (2): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=256, out_features=64, b

In [17]:
optimizer = optim.AdamW(model.parameters(), lr=lr)
scaler = amp.GradScaler() if use_torch_amp else None

scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, ep-1)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

rce_best = 0    

## start training

In [17]:
print(model_name)

MF_len48_joint_thr25_3weeks


In [19]:
for epoch in range(1, ep+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler_warmup.step(epoch-1) 
    
    # 5 parts per epoch
    idx_this_ep = train_parts_order[(epoch*5-5):epoch*5]
    
    train_lst = []
    for idx in idx_this_ep:
        train_lst.append(read_norm_merge(PATHS[idx], 'train' if idx<10 else 'both').to_pandas())
    train = pd.concat(train_lst)
 
    gc.collect();gc.collect();
    
    train_dataset = AllDataset(train, max_len_txt, NUMERIC_COLUMNS, CAT_COLUMNS)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, drop_last=True) 
    
    train_loss = train_epoch(model, train_loader, optimizer, scaler)
    valid_loss,rce,mean_rce = valid_epoch(model, valid_loader)
   
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.4f}, valid loss: {valid_loss:.4f}, mean_rce: {mean_rce:.2f}'
    for col in ['retweet', 'reply',  'like', 'retweet_comment']:
        content += f', {col}: {rce[col]:.2f}'
        
    print(content)
    
    if mean_rce > rce_best:
        print('rce_best increased ({:.6f} --> {:.6f}).  Saving model ...'.format(rce_best, mean_rce))
        rce_best = mean_rce
                
        torch.save(model.state_dict(), f'../models/{model_name}_best.pth')
        
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict() if scaler else None,      
            'rce_best': rce_best,
        },
        f'../models/{model_name}_last.pth'
    )            
        
torch.save(model.state_dict(), f'../models/{model_name}_final.pth')        

Sun May 30 00:24:52 2021 Epoch: 1


100%|██████████| 10083/10083 [02:00<00:00, 83.97it/s]


Sun May 30 01:19:18 2021 Epoch 1, lr: 0.0001000, train loss: 0.2837, valid loss: 0.2565, mean_rce: 8.31, retweet: 9.04, reply: 11.18, like: 10.27, retweet_comment: 2.74
rce_best increased (0.000000 --> 8.305332).  Saving model ...
Sun May 30 01:21:23 2021 Epoch: 2


MemoryError: std::bad_alloc: CUDA error at: /home/bo/anaconda3/envs/rapids19/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

### change to pd loading

In [18]:
        sd = torch.load(f'../models/{model_name}_last.pth', map_location='cpu')
        from_epoch = sd['epoch']
        sd['model_state_dict'] = {k[7:] if k.startswith('module.') else k: sd['model_state_dict'][k] for k in sd['model_state_dict'].keys()}
        model.load_state_dict(sd['model_state_dict'], strict=True)
        optimizer.load_state_dict(sd['optimizer_state_dict'])
        scaler.load_state_dict(sd['scaler_state_dict'])

In [19]:
for epoch in range(1, ep+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler_warmup.step(epoch-1) 
    
    if epoch<=1: continue
    
    # 5 parts per epoch
    idx_this_ep = train_parts_order[(epoch*5-5):epoch*5]
    
    train_lst = []
    for idx in tqdm(idx_this_ep):
        train_lst.append(read_norm_merge(PATHS[idx], 'train' if idx<10 else 'both'))
    train = pd.concat(train_lst)
 
    gc.collect();gc.collect();
    
    train_dataset = AllDataset(train, max_len_txt, NUMERIC_COLUMNS, CAT_COLUMNS)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, drop_last=True) 
    
    train_loss = train_epoch(model, train_loader, optimizer, scaler)
    valid_loss,rce,mean_rce = valid_epoch(model, valid_loader)
   
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.4f}, valid loss: {valid_loss:.4f}, mean_rce: {mean_rce:.2f}'
    for col in ['retweet', 'reply',  'like', 'retweet_comment']:
        content += f', {col}: {rce[col]:.2f}'
        
    print(content)
    
    if mean_rce > rce_best:
        print('rce_best increased ({:.6f} --> {:.6f}).  Saving model ...'.format(rce_best, mean_rce))
        rce_best = mean_rce
                
        torch.save(model.state_dict(), f'../models/{model_name}_best.pth')
        
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict() if scaler else None,      
            'rce_best': rce_best,
        },
        f'../models/{model_name}_last.pth'
    )            
        
torch.save(model.state_dict(), f'../models/{model_name}_final.pth')        

  0%|          | 0/5 [00:00<?, ?it/s]

Sun May 30 01:38:04 2021 Epoch: 1
Sun May 30 01:38:04 2021 Epoch: 2


100%|██████████| 10083/10083 [02:05<00:00, 80.60it/s]


Sun May 30 02:41:33 2021 Epoch 2, lr: 0.0010000, train loss: 0.2498, valid loss: 0.2447, mean_rce: 11.53, retweet: 13.58, reply: 13.88, like: 14.72, retweet_comment: 3.94
rce_best increased (0.000000 --> 11.530009).  Saving model ...


  0%|          | 0/5 [00:00<?, ?it/s]

...

Tue Jun  1 02:41:56 2021 Epoch: 45


100%|██████████| 10083/10083 [02:10<00:00, 77.24it/s]


Tue Jun  1 03:46:35 2021 Epoch 45, lr: 0.0000049, train loss: 0.2024, valid loss: 0.1980, mean_rce: 26.05, retweet: 30.50, reply: 26.39, like: 32.02, retweet_comment: 15.30
Tue Jun  1 03:48:07 2021 Epoch: 46


100%|██████████| 10083/10083 [02:10<00:00, 77.51it/s]


Tue Jun  1 04:56:12 2021 Epoch 46, lr: 0.0000012, train loss: 0.2025, valid loss: 0.1979, mean_rce: 26.04, retweet: 30.50, reply: 26.37, like: 32.05, retweet_comment: 15.24


In [ ]:
# 20 parts, 5 epochs
mean_rce: 16.51, retweet: 20.27, reply: 18.26, like: 20.17, retweet_comment: 7.35
                    
Epoch 20, lr: 0.0000794, train loss: 0.2142, valid loss: 0.2233, 
mean_rce: 18.66, retweet: 23.00, reply: 20.36, like: 21.72, retweet_comment: 9.56     
                    
Epoch 35, lr: 0.0001654, train loss: 0.2105, valid loss: 0.2220, 
mean_rce: 19.25, retweet: 23.58, reply: 21.08, like: 22.09, retweet_comment: 10.27                    

# xgb feat NN                    
mean_rce: 20.25, retweet: 23.39, reply: 19.07, like: 13.02, retweet_comment: 25.54                    

## load best ep and inference

In [27]:
sd = torch.load(f'../models/{model_name}_best.pth')
sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
model.load_state_dict(sd, strict=True)

<All keys matched successfully>

In [28]:
label_names = sorted(label_names)
label_names

['like', 'reply', 'retweet', 'retweet_comment']

In [29]:
model.eval()
val_loss = []
LOGITS = []
TARGETS = []
with torch.no_grad():
    for batch in tqdm(valid_loader):
        x_cat, x_cont, text_tok, targets = batch
        x_cat = x_cat.cuda()     
        x_cont = x_cont.cuda()
        text_tok = text_tok.cuda()
        targets = targets.cuda()            
        logits = model(x_cat, x_cont, text_tok)
        loss = criterion(logits, targets)
        val_loss.append(loss.item())
        LOGITS.append(logits.cpu())
        TARGETS.append(targets.cpu())

LOGITS = torch.cat(LOGITS)
TARGETS = torch.cat(TARGETS)
rce = {}
for i in range(4):
    rce[label_names[i]] = compute_rce_fast(cp.asarray(LOGITS[:,i].sigmoid()),cp.asarray(TARGETS[:,i])).get()            
mean_rce = np.mean([v for k,v in rce.items()])
mean_rce

100%|██████████| 10083/10083 [01:43<00:00, 97.81it/s] 


20.13601

In [30]:
# df_quantile = pd.concat([pd.read_parquet(path)[['quantile']] for path in VALID_PATHS]).reset_index(drop=True)
# df_quantile = df_quantile.apply(np.expm1).round().astype(int)
df_quantile = valid[['quantile']].copy().reset_index(drop=True)
df_quantile.shape

yquantile = cupy.asarray(df_quantile.values)
oof = cupy.asarray(LOGITS.sigmoid())
yvalid = cupy.asarray(TARGETS)

In [31]:
from util import compute_prauc, average_precision_score,display_score

rce_output = {}
ap_output = {}
for i in range(4):
    prauc_out = []
    rce_out = []
    ap_out = []
    for j in range(5):
        this_quantile_idx = (df_quantile == j)['quantile'].values
        yvalid_tmp = yvalid[this_quantile_idx][:, i]
        oof_tmp = oof[this_quantile_idx][:, i]
        prauc = compute_prauc(oof_tmp, yvalid_tmp)
        rce   = compute_rce_fast(oof_tmp, yvalid_tmp).item()
        ap    = average_precision_score(cupy.asnumpy(yvalid_tmp),cupy.asnumpy(oof_tmp))
        prauc_out.append(prauc)
        rce_out.append(rce)
        ap_out.append(ap)
    rce_output[label_names[i]] = rce_out
    ap_output[label_names[i]] = ap_out

In [32]:
print(model_name)
display_score(rce_output, ap_output)

gru_cat5_cont36_frzemb768_gru128_len64_thr50_3weeks
Quantile Group|AP Retweet|RCE Retweet|  AP Reply|  RCE Reply|   AP Like|   RCE Like|AP RT comment|RCE RT comment
        0          0.4827     25.3136     0.2257     18.1147     0.7328     17.5110     0.0545      9.7572
        1          0.4643     24.6773     0.2055     18.0764     0.7295     17.6294     0.0554     10.1372
        2          0.4435     23.6253     0.2155     18.7955     0.7326     18.1288     0.0489      8.8695
        3          0.4352     23.1426     0.2309     19.5610     0.7339     18.9700     0.0433      8.8682
        4          0.4320     24.3085     0.2189     21.0982     0.7610     27.1258     0.0494     11.0005
     Average       0.4515     24.2134     0.2193     19.1291     0.7379     19.8730     0.0503      9.7265


In [28]:
# XGB
%%time
display_score(rce_output, ap_output)

Quantile Group|AP Retweet|RCE Retweet|  AP Reply|  RCE Reply|   AP Like|   RCE Like|AP RT comment|RCE RT comment
        0          0.4767     23.8403     0.2466     19.7694     0.7496     18.7935     0.0803     11.3031
        1          0.4608     24.3840     0.2391     20.9897     0.7444     20.0392     0.0680     10.6829
        2          0.4501     24.8955     0.2513     22.2341     0.7393     20.2754     0.0682     11.2699
        3          0.4384     24.5673     0.2677     23.6996     0.7334     20.9961     0.0662     11.5356
        4          0.4124     24.7017     0.2411     24.6334     0.7059     20.6678     0.0710     15.1545
     Average       0.4477     24.4778     0.2492     22.2652     0.7345     20.1544     0.0707     11.9892
CPU times: user 0 ns, sys: 1 ms, total: 1 ms
Wall time: 950 µs


## load best ep and inference LB valid

In [12]:
def read_norm_merge(ddf):

    ddf['quantile'] = 0
    quantiles = [ 240,  588, 1331, 3996]
    for i, quant in enumerate(quantiles):
        ddf['quantile'] = (ddf['quantile']+(ddf['a_follower_count']>quant).astype('int8')).astype('int8')

    ddf['date'] = cudf.to_datetime(ddf['timestamp'], unit='s')
       
    ddf['a_ff_rate'] = (ddf['a_following_count'] / ddf['a_follower_count']).astype('float32')
    ddf['b_ff_rate'] = (ddf['b_follower_count']  / ddf['b_following_count']).astype('float32')
    ddf['ab_fing_rate'] = (ddf['a_following_count'] / ddf['b_following_count']).astype('float32')
    ddf['ab_fer_rate'] = (ddf['a_follower_count'] / (1+ddf['b_follower_count'])).astype('float32')
    ddf['a_age'] = ddf['a_account_creation'].astype('int16') + 128
    ddf['b_age'] = ddf['b_account_creation'].astype('int16') + 128
    ddf['ab_age_dff'] = ddf['b_age'] - ddf['a_age']
    ddf['ab_age_rate'] = ddf['a_age']/(1+ddf['b_age'])

    ## Normalize
    for col in NUMERIC_COLUMNS:
        if col == 'tw_len_quest':
            ddf[col] = np.clip(ddf[col].values.get(),0,None)
        if ddf[col].dtype == 'uint16':
            ddf[col].astype('int32')

        if col == 'ab_age_dff':
            ddf[col] = ddf[col] / 256.            
        elif 'int' in str(ddf[col].dtype) or 'float' in str(ddf[col].dtype):    
            ddf[col] = np.log1p(ddf[col])

        if ddf[col].dtype == 'float64':
            ddf[col] = ddf[col].astype('float32') 

    ## get categorical embedding id        
    for col in CAT_COLUMNS:
        ddf[col] = ddf[col].astype('float')
        if col in ['a_user_id','b_user_id']:
            mapping_col = 'a_user_id_b_user_id'
        else:
            mapping_col = col
        mapping = cudf.read_parquet(f'/raid/recsys_pre_TE_w_tok/workflow_232parts_joint_thr25/categories/unique.{mapping_col}.parquet').reset_index()
        mapping.columns = ['index',col]
        ddf = ddf.merge(mapping, how='left', on=col).drop(columns=[col]).rename(columns={'index':col})
        ddf[col] = ddf[col].fillna(0).astype('int')        

    label_names = ['reply', 'retweet', 'retweet_comment', 'like']
    DONT_USE = ['timestamp','a_account_creation','b_account_creation','engage_time',
                'fold', 'dt_dow', 'a_account_creation', 
                'b_account_creation', 'elapsed_time', 'links','domains','hashtags','id', 'date', 'is_train', 
                'tw_hash0', 'tw_hash1', 'tw_hash2', 'tw_http0', 'tw_uhash', 'tw_hash', 'tw_word0', 
                'tw_word1', 'tw_word2', 'tw_word3', 'tw_word4', 'dt_minute', 'dt_second',
               'dt_day', 'group', 'text', 'tweet_id', 'tw_original_user0', 'tw_original_user1', 'tw_original_user2',
                'tw_rt_user0', 'tw_original_http0', 'tw_tweet',]
    DONT_USE = [c for c in ddf.columns if c in DONT_USE]
    gc.collect(); gc.collect()
    
    return ddf.drop(columns=DONT_USE)

In [13]:
%%time
df = cudf.read_parquet('/raid/recsys_valid/valid_proc.parquet',num_rows=7_000_000)
df = read_norm_merge(df).to_pandas()

df2 = cudf.read_parquet('/raid/recsys_valid/valid_proc.parquet',skiprows=7_000_000)
df2 = read_norm_merge(df2).to_pandas()

valid = pd.concat([df,df2])
del df,df2
gc.collect()

valid_dataset = AllDataset(valid, max_len_txt, NUMERIC_COLUMNS, CAT_COLUMNS)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers) 
valid.shape, len(valid_loader)

CPU times: user 17.4 s, sys: 10.5 s, total: 27.9 s
Wall time: 30 s


((14461760, 47), 14123)

In [14]:
# sd = torch.load(f'../models/{model_name}_best.pth')
sd = torch.load('/home/bo/kaggle/recsys/recsysChallenge2021/bo/sub/v11_len48_thr25_joint_MF/MF_len48_joint_thr25_3weeks_best.pth')
sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
model.load_state_dict(sd, strict=True)

<All keys matched successfully>

In [15]:
label_names = sorted(label_names)
label_names

['like', 'reply', 'retweet', 'retweet_comment']

In [16]:
model.eval()
val_loss = []
LOGITS = []
TARGETS = []
with torch.no_grad():
    for batch in tqdm(valid_loader):
        x_cat, x_cont, text_tok, targets = batch
        x_cat = x_cat.cuda()     
        x_cont = x_cont.cuda()
        text_tok = text_tok.cuda()
        targets = targets.cuda()            
        logits = model(x_cat, x_cont, text_tok)
        loss = criterion(logits, targets)
        val_loss.append(loss.item())
        LOGITS.append(logits.cpu())
        TARGETS.append(targets.cpu())

LOGITS = torch.cat(LOGITS)
TARGETS = torch.cat(TARGETS)
rce = {}
for i in range(4):
    rce[label_names[i]] = compute_rce_fast(cp.asarray(LOGITS[:,i].sigmoid()),cp.asarray(TARGETS[:,i])).get()            
mean_rce = np.mean([v for k,v in rce.items()])
mean_rce

100%|██████████| 14123/14123 [02:46<00:00, 84.59it/s]


13.316917

In [17]:
# df_quantile = pd.concat([pd.read_parquet(path)[['quantile']] for path in VALID_PATHS]).reset_index(drop=True)
# df_quantile = df_quantile.apply(np.expm1).round().astype(int)
df_quantile = valid[['quantile']].copy().reset_index(drop=True)
df_quantile.shape

yquantile = cupy.asarray(df_quantile.values)
oof = cupy.asarray(LOGITS.sigmoid())
yvalid = cupy.asarray(TARGETS)

In [18]:
from util import compute_prauc, average_precision_score,display_score

rce_output = {}
ap_output = {}
for i in range(4):
    prauc_out = []
    rce_out = []
    ap_out = []
    for j in range(5):
        this_quantile_idx = (df_quantile == j)['quantile'].values
        yvalid_tmp = yvalid[this_quantile_idx][:, i]
        oof_tmp = oof[this_quantile_idx][:, i]
        prauc = compute_prauc(oof_tmp, yvalid_tmp)
        rce   = compute_rce_fast(oof_tmp, yvalid_tmp).item()
        ap    = average_precision_score(cupy.asnumpy(yvalid_tmp),cupy.asnumpy(oof_tmp))
        prauc_out.append(prauc)
        rce_out.append(rce)
        ap_out.append(ap)
    rce_output[label_names[i]] = rce_out
    ap_output[label_names[i]] = ap_out

In [19]:
# public test
print(model_name)
display_score(rce_output, ap_output)

MF_len48_joint_thr25_3weeks
Quantile Group|AP Retweet|RCE Retweet|  AP Reply|  RCE Reply|   AP Like|   RCE Like|AP RT comment|RCE RT comment
        0          0.3648     19.1185     0.1768     17.4406     0.5983      8.8159     0.0343      8.4536
        1          0.3457     18.4016     0.1811     17.8598     0.5753      6.9555     0.0309      8.3305
        2          0.3388     17.9507     0.2000     18.7393     0.5646      6.6237     0.0306      8.0775
        3          0.3504     17.6180     0.2199     19.9633     0.5772      7.1854     0.0310      8.4171
        4          0.3247     16.5028     0.1267     14.6986     0.6501     10.9945     0.0286      8.7294
     Average       0.3449     17.9183     0.1809     17.7403     0.5931      8.1150     0.0311      8.4016


In [ ]:
boliu0	sub_0601   0.3446	17.9344     	0.1829	17.8545	    0.5926	    8.0979	    0.0314	   8.4308	9 hours	258

In [31]:
print(model_name)
display_score(rce_output, ap_output)

MF_len48_joint_thr25_3weeks
Quantile Group|AP Retweet|RCE Retweet|  AP Reply|  RCE Reply|   AP Like|   RCE Like|AP RT comment|RCE RT comment
        0          0.5608     32.5540     0.2911     23.4060     0.8070     28.8335     0.0951     15.3339
        1          0.5250     30.9199     0.2968     24.7680     0.8037     29.8295     0.0780     14.2486
        2          0.5138     30.4446     0.3222     26.5348     0.7998     30.7027     0.0694     14.0940
        3          0.5267     31.0953     0.3355     27.6502     0.8038     31.8786     0.0766     15.1034
        4          0.4854     29.5893     0.2521     24.8843     0.7972     33.1890     0.0790     15.7639
     Average       0.5223     30.9206     0.2995     25.4487     0.8023     30.8866     0.0797     14.9087


In [25]:
print(model_name)
display_score(rce_output, ap_output)

MF_len48_joint_thr25_3weeks
Quantile Group|AP Retweet|RCE Retweet|  AP Reply|  RCE Reply|   AP Like|   RCE Like|AP RT comment|RCE RT comment
        0          0.5756     33.0314     0.2987     22.8756     0.8081     28.1401     0.0998     15.3117
        1          0.5540     32.3156     0.2850     23.5546     0.8063     29.1117     0.0939     15.5322
        2          0.5309     31.2906     0.2963     24.6103     0.8050     29.6993     0.0826     14.4356
        3          0.5168     30.4555     0.3086     25.7036     0.8003     30.3232     0.0723     14.2019
        4          0.4954     29.9846     0.2819     26.2819     0.7986     32.8644     0.0774     15.5282
     Average       0.5345     31.4155     0.2941     24.6052     0.8037     30.0277     0.0852     15.0019
